# Train model

In [ ]:
import csv
import os
import pandas as pd
import warnings
import numpy as np
import tensorflow as tf
import numpy as np
import os
import time

In [ ]:
#road file corpus
df_corpus = pd.read_csv('[PATH_OF_FILE_df_cospus.csv]' )

#df_corpus to list
corpusList = df_corpus['corpus'].tolist()


vocab = sorted(set(corpusList))
print('Corpus length (in words):', len(corpusList))
print('Unique words in corpus: {}'.format(len(vocab)))
word2idx = {u: i for i, u in enumerate(vocab)}
idx2words = np.array(vocab)
word_as_int = np.array([word2idx[c] for c in corpusList])


# The maximum length sentence we want for a single input in words
seqLength = 8
examples_per_epoch = len(corpusList)//(seqLength + 1) # number of seqLength+1 sequences in the corpus

# Create training / targets batches
wordDataset = tf.data.Dataset.from_tensor_slices(word_as_int)
print(wordDataset)
sequencesOfWords = wordDataset.batch(seqLength + 1, drop_remainder=True) # generating batches of 8 words each, typically converting list of words (sequence) to string
print(sequencesOfWords)

In [ ]:
def split_input_target(chunk): # This is where right shift happens
  input_text = chunk[:-1]
  print(input_text)
  target_text = chunk[1:]
  print(target_text)
  return input_text, target_text # returns training and target sequence for each batch

dataset = sequencesOfWords.map(split_input_target) # dataset now contains a training and a target sequence for each 8 word slice of the corpus


#define BATCH_SIZE and BUFFER_SIZE
BATCH_SIZE = 64 # each batch contains 64 sequences. Each sequence contains 8 words (seqLength)
BUFFER_SIZE = 100 # Number of batches that will be processed concurrently
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

# Length of the vocabulary in words
vocab_size = len(vocab)
# The embedding dimension
embedding_dim = 256
# Number of GRU units
rnn_units = 1024

def createModel(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = createModel(vocab_size = len(vocab), embedding_dim=embedding_dim, rnn_units=rnn_units, batch_size=BATCH_SIZE)

def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss)

checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}") 

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

#Train Model
EPOCHS = 20
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])
tf.train.latest_checkpoint(checkpoint_dir)
model = createModel(len(vocab), embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))
model.summary()



def generateLyrics(model, startString, temp,num_generate):
  print("---- Generating lyrics starting with '" + startString + "' ----")
  # Number of words to generate
  num_generate = num_generate

  # Converting our start string to numbers (vectorizing)
  start_string_list =  [w for w in startString.split(' ')]
  input_eval = [word2idx[s] for s in start_string_list]
  input_eval = tf.expand_dims(input_eval, 0)

  text_generated = []

  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # temp represent how 'conservative' the predictions are. 
      # Lower temp leads to more predictable (or correct) lyrics
      predictions = predictions / temp 
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)
      text_generated.append(' ' + idx2words[predicted_id])

  return (startString + ''.join(text_generated))


#save trained model for future use (so we do not have to train it every time we want to generate text)
model.save('model_generate_song.h5')

# Generate model

In [4]:
import csv
import os
import pandas as pd
import warnings
import numpy as np
import tensorflow as tf
import numpy as np
import os
import time

In [6]:
#road file corpus
# df_corpus = pd.read_csv('[PATH_OF_FILE_df_cospus.csv]' )
df_corpus = pd.read_csv('dataset_corpus_lyrics.csv')

#df_corpus to list
corpusList = df_corpus['corpus'].tolist()


vocab = sorted(set(corpusList))
print('Corpus length (in words):', len(corpusList))
print('Unique words in corpus: {}'.format(len(vocab)))
word2idx = {u: i for i, u in enumerate(vocab)}
idx2words = np.array(vocab)
word_as_int = np.array([word2idx[c] for c in corpusList])

Corpus length (in words): 314907
Unique words in corpus: 7125


In [7]:
def generateLyrics(model, startString, temp,num_generate):
  print("---- Generating lyrics starting with '" + startString + "' ----")
  # Number of words to generate
  num_generate = num_generate

  # Converting our start string to numbers (vectorizing)
  start_string_list =  [w for w in startString.split(' ')]
  input_eval = [word2idx[s] for s in start_string_list]
  input_eval = tf.expand_dims(input_eval, 0)

  text_generated = []

  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # temp represent how 'conservative' the predictions are. 
      # Lower temp leads to more predictable (or correct) lyrics
      predictions = predictions / temp 
      # print(predictions)
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
      # print(predicted_id)

      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)
      # print(input_eval)

      text_generated.append(' ' + idx2words[predicted_id])
      

  return (startString + ''.join(text_generated))

In [8]:
# load Model
from keras.models import load_model
# model_w = load_model('[PATH_OF_MODEL_model_generate_song.h5]')
model_w = load_model('model_generate_song.h5')
#save trained model for future use (so we do not have to train it every time we want to generate text)

In [9]:
print(generateLyrics(model_w, startString="คิดถึง", temp=0.6,num_generate = 100))

# while (True):
#   print('Enter start string:')
#   input_str = input().lower().strip()
#   temp=0.6
#   num_generate = 100
#   print(generateLyrics(model_w, startString=input_str, temp=temp,num_generate=num_generate))
  

---- Generating lyrics starting with 'คิดถึง' ----
คิดถึง ทีไร แค่นี้ เอง ก็ รู้ มัน อยู่ ใกล้ ใจ เธอ เป็น ของ ฉัน รู้ ว่า มัน ไม่ ดี กับ เธอ แม้ เรา มี เรา กัน ทุกอย่าง แค่ เพียง วูบ เดียว มา แค่ เพียง ชั่วคราว ผ่าน เข้ามา แล้ว ทำ ฉัน ต้อง เสีย ใจมา แค่ เพียง วูบ เดียว มา วูบ เดียว แค่ จาก ผ่าน เข้ามา แล้ว ทำให้ น้ำตา ริน ไหล ไม่ รู้ ว่า นาน เท่าไหร่ ที่จะ หลุดพ้น กับ ความรัก ที่ ทำให้ หมองหม่น เหลือเกิน แต่ ผ่าน มา เพียง วูบ เดียว มา แค่ เพียง ชั่วคราว ผ่าน เข้ามา แล้ว ทำ ฉัน ต้อง เสีย ใจมา แค่ เพียง วูบ เดียว มา วูบ เดียว แค่ จาก ผ่าน เข้ามา


In [1]:
print(generateLyrics(model_w, startString="พ่อ", temp=0.6,num_generate = 50))

NameError: name 'generateLyrics' is not defined

In [11]:
print(generateLyrics(model_w, startString="แม่", temp=0.6,num_generate = 100))

---- Generating lyrics starting with 'แม่' ----
แม่ ง นี่ ของจริง มัน ไม่ ใช่ อยาก จะ หนี ไป กับ จานบิน อยู่ บน โลก ใบ นี้ เปลี่ยน ให้ เธอ ได้ เห็น ทั้งที่ เธอ ยัง เหมือนเดิม อีก เท่าไหร่ เรื่อง เรา ไม่ ไป ไหน แม้กระทั่ง ตอนนี้ เขา ยังอยู่ ตรงนั้น ใน หัวใจ แม้ จะ นาน เท่าไหร่ ยัง รัก เธอ ตื่น จาก ภาพลวงตา แต่ ไม่ มี เธอ เคียง กาย เหมือน เป็น ดั่ง ความฝัน อาจ เป็น ดั่ง เช่นเคย ฉัน ยืน อยู่ ตรงนี้ ที่ หัวใจ ต้องการ สุดท้าย มัน กลายเป็น หลุมดำ แล้ว ฉัน คง ทำได้ แค่ เพียง มองดู เธอ ค่อยๆ ทรมาน ต่อไป เป็น กำลังใจ ให้ เธอ รู้ ว่า โอ้ เธอ นะ คนดี ได้ โปรด อย่า จาก
